In [1]:
!pip install -q datasets pytorch-lightning transformers evaluate sacrebleu accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.6/720.6 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Загрузка датасета и предобработка

In [2]:
import torch
import torch.nn as nn

from typing import List, Optional

from datasets import load_dataset
from tokenizers.processors import TemplateProcessing
from tokenizers import Tokenizer
from tokenizers.models import BPE

from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

from transformers import PreTrainedTokenizerFast, PreTrainedTokenizer

In [3]:
books = load_dataset("opus_books", "en-fr")
books["train"] = books["train"].select(range(10000))

books = books.filter(lambda x: len(x['translation']['en']) < 250)
print(books)

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Dataset opus_books downloaded and prepared to /root/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 8919
    })
})


In [4]:
books['train'][0]

{'id': '0', 'translation': {'en': 'The Wanderer', 'fr': 'Le grand Meaulnes'}}

In [5]:
ALL_SETENCES_FILE = 'all_book_sentences.txt'

with open(ALL_SETENCES_FILE, 'w') as f:
    for item in books['train']:
        f.write(item['translation']['en'] + "\n")
        f.write(item['translation']['fr'] + "\n")

In [6]:
!head all_book_sentences.txt

The Wanderer
Le grand Meaulnes
Alain-Fournier
Alain-Fournier
First Part
PREMIÈRE PARTIE
I
CHAPITRE PREMIER
THE BOARDER
LE PENSIONNAIRE


In [7]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

VOCAB_SIZE = 20000

bpe_trainer = BpeTrainer(special_tokens=["[UNK]", "[BOS]", "[EOS]", "[PAD]"], show_progress=True, vocab_size=VOCAB_SIZE)

tokenizer.pre_tokenizer = Whitespace()

files = [ ALL_SETENCES_FILE ]

tokenizer.train(files, bpe_trainer)

In [8]:
tokenizer.post_processor = TemplateProcessing(
    single="[BOS] $A [EOS]",
    special_tokens=[
        ("[BOS]", tokenizer.token_to_id("[BOS]")),
        ("[EOS]", tokenizer.token_to_id("[EOS]")),
    ],
)

tokenizer.save("tokenizer.json")

In [9]:
tokenizer.encode('[BOS]')

Encoding(num_tokens=3, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [10]:
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)
fast_tokenizer.bos_token = "[BOS]"
fast_tokenizer.eos_token = "[EOS]"
fast_tokenizer.pad_token = "[PAD]"
fast_tokenizer.unk_token = "[UNK]"

In [11]:
source_lang = "en"
target_lang = "fr"

def preprocess_function(examples):
    inputs = [example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = fast_tokenizer(inputs, text_target=targets, max_length=64, truncation=True, add_special_tokens=True)
    return model_inputs


In [12]:
books_preprocessed = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/8919 [00:00<?, ? examples/s]

In [13]:
books_preprocessed['train'][0]

{'id': '0',
 'translation': {'en': 'The Wanderer', 'fr': 'Le grand Meaulnes'},
 'input_ids': [1, 281, 19049, 1203, 2],
 'token_type_ids': [0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1],
 'labels': [1, 521, 558, 346, 2]}

In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=fast_tokenizer, return_tensors="pt")

In [15]:
fast_tokenizer("test", text_target="test")

{'input_ids': [1, 4207, 2], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1], 'labels': [1, 4207, 2]}

In [16]:
fast_tokenizer.eos_token_id

2

In [17]:

data_collator( [ {"input_ids": [ 100, 200, 300 ], "labels": [100,200,300, 400, 500]}, { "input_ids": [ 100, 200, 300,400,500], "labels": [100,200,300,400,500,600]} ] )


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[100, 200, 300,   3,   3],
        [100, 200, 300, 400, 500]]), 'labels': tensor([[ 100,  200,  300,  400,  500, -100],
        [ 100,  200,  300,  400,  500,  600]]), 'attention_mask': tensor([[1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1]])}

In [58]:
# ВАЖНО! Эту ячейку в пайплайнах будет копировать в отдельный файл.
# Нельзя изменять названия и конструкторы классов.
# Не стоит изменять сигнатуры методов
# Не стоит добавлять сюда новые ненужные импорты
# Не стоит удалять импорты, которые тут были
# Не стоит добавлять в эту ячейку новые классы

from transformers.modeling_outputs import Seq2SeqModelOutput, Seq2SeqLMOutput

import torch
import torch.nn as nn

from transformers.modeling_outputs import Seq2SeqModelOutput, Seq2SeqLMOutput

from transformers import GenerationConfig, PretrainedConfig, PreTrainedModel
import random
import numpy as np

# Для прохождения тестов не будет требоваться, чтобы модель полностью обучилась
# это может занять много времени. В этой домашке будет достаточно переобучить модель
# на небольшой части датасета.
#
# Цель домашки -- это реализовать свой трансформер
# Нужно дополнить процесс обучения ниже, заполнить пропуски
#
# Чтобы домашка была тестируемой, пришлось прибегнуть к таким требованиям
# Кроме того, нет большого практического смысла в обучении такой архитектуры,
# потому что она устарела, лучше обучите трансформер во втором ноутбуке)
#
# HINT!
# Прописывайте размерности разных тензоров в комментах -- так будет проще разбираться в коде и дебажить
#
# Черпать вдохновение можно отсюда http://nlp.seas.harvard.edu/2018/04/03/attention.html
# И Attention Is All You Need https://arxiv.org/abs/1706.03762

# PretrainedConfig см тут https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/configuration#transformers.PretrainedConfig
class TransformerAttentionConfig(PretrainedConfig):
    model_type = "rnn"

    r"""
    В классе конфига должны быть описаны все гипер-параметры модели

    Args:
        vocab_size (`int`):
            Размер словаря
        embedding_dim (`int`):
            Размерность эмбэддингов
        hidden_dim (`int`):
            размерность скрытых слоев
        num_layers (`int`):
            количество слоев трансформера (и для енкодера, и для декодера)
        max_length (`int`):
            Максимальная длинна сгенерированной последовательности

    """

    def __init__(
        self,
        vocab_size=20000,
        embedding_dim=128,
        hidden_dim=128,
        num_layers=3,
        max_length=64,
        # Эти токены должны быть предопределены уже в PretrainedConfig
        # https://github.com/huggingface/transformers/blob/cf11493dce0a1d22446efe0d6c4ade02fd928e50/src/transformers/configuration_utils.py#L214
        # pad_token_id=None,
        # bos_token_id=None,
        # eos_token_id=None,
        **kwargs,
    ):
        super().__init__(**kwargs, max_length=max_length)

        assert embedding_dim == hidden_dim, 'this transformer implementation requires embedding_dim to be equals to hidden_dim'

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers


# вот тут про PreTrainedModel  https://github.com/huggingface/transformers/blob/cf11493dce0a1d22446efe0d6c4ade02fd928e50/src/transformers/modeling_utils.py#LL1009C7-L1009C22
class Seq2SeqTransformerAttention(PreTrainedModel):

    config_class = TransformerAttentionConfig
    base_model_prefix = "transformer"
    supports_gradient_checkpointing = False

    def __init__(self, config):

        super().__init__( config )

        self.generation_config = GenerationConfig()

        # начинаем с того, что правильно опишем используемые модули
        # используем self.config.*

        self.embeddings = nn.Embedding(num_embeddings=self.config.vocab_size,
                                       embedding_dim=self.config.embedding_dim)
        self.embeddings_dropout = nn.Dropout(p=0.1)

        self.encoder_norm = nn.LayerNorm(self.config.embedding_dim)

        encoder_layer = nn.TransformerEncoderLayer(d_model=self.config.embedding_dim,
                                                   nhead=8, batch_first=True)
        encoder_norm = nn.LayerNorm(self.config.embedding_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer,
                                                         num_layers=self.config.num_layers)

        decoder_layer = nn.TransformerDecoderLayer(d_model=self.config.embedding_dim,
                                                   nhead=8, batch_first=True)
        decoder_norm = nn.LayerNorm(self.config.embedding_dim)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer,
                                                         num_layers=self.config.num_layers)

        self.decoder_labels_linear = nn.Linear(self.config.embedding_dim, self.config.vocab_size)

        self.criterion = nn.CrossEntropyLoss(ignore_index=self.config.pad_token_id)

        return


    # copy paste from
    # http://nlp.seas.harvard.edu/2018/04/03/attention.html
    def subsequent_mask(self, size, device='cpu'):
        "Mask out subsequent positions."
        attn_shape = (size, size)
        subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
        return (torch.from_numpy(subsequent_mask) != 0).to(device)

    def encode(self, input_embeddings=None, key_padding_mask=None):

        # получаем неконтекстные эмбэддинги
        # получаем контекстные эмбэддинги с помощью self.encoder

        encoder_embeddings = self.transformer_encoder(input_embeddings, key_padding_mask)

        return encoder_embeddings

    def decode(self, encoder_outputs=None, lebels_embeddings=None, key_padding_mask=None, encoder_key_padding_mask=None):
        # не забудьте, что нужно обработать сдвиг токенов
        # декодер должен вернуть для каждого текущего токена последующий токен
        # у последнего токена нет последующего тк он последний
        # поэтому последний токен не надо передавать

        # прелесть трансформеров заключается в том, что у них параллелится обучение декодера
        # не надо на каждый токен запускать generate_encoded как это было в RNN
        # это возможно благодаря тому, что мы можем замаскировать будущие токены в механизме внимания
        # декодера с помощью subsequent_mask
        tgt_mask = self.subsequent_mask( lebels_embeddings.shape[1] - 1, device=encoder_outputs.device)

        # не забываем про сдвиг
        if key_padding_mask is not None:
          tgt_key_padding_mask = key_padding_mask[:-1]
        else:
          tgt_key_padding_mask = key_padding_mask
        if encoder_key_padding_mask is not None:
          memory_key_padding_mask = encoder_key_padding_mask[:-1]
        else:
          memory_key_padding_mask = encoder_key_padding_mask

        # см доку https://pytorch.org/docs/stable/generated/torch.nn.TransformerDecoder.html
        # осталось правильно передать аргументы
        # не забываем про сдвиг!
        decoder_output = self.transformer_decoder(tgt=lebels_embeddings[1:],
                                                  memory=encoder_outputs[:-1],
                                                  tgt_mask=tgt_mask[:-1],
                                                  tgt_key_padding_mask=tgt_key_padding_mask,
                                                  memory_key_padding_mask=memory_key_padding_mask)

        return decoder_output

    # https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/output#transformers.modeling_outputs.Seq2SeqModelOutput
    def forward(self, input_ids=None, labels=None, attention_mask=None, token_type_ids=None, ) -> Seq2SeqModelOutput:

        # input_ids: [ batch_size, src_seq_len ]
        # labels: [ batch_size, tgt_seq_len ]

        encoder_key_padding_mask = (attention_mask == 0)

        decoder_key_padding_mask = (labels == -100)

        input_embeddings = self.embeddings(input_ids)
        input_embeddings = self.embeddings_dropout(input_embeddings)

        labels[labels == -100] = self.config.pad_token_id
        lebels_embeddings = self.embeddings(labels)
        lebels_embeddings = self.embeddings_dropout(lebels_embeddings) # используется для обучения декодера

        # encoder_outputs может еще называться memory
        # TODO размерность [ ? ]
        encoder_outputs = self.encode(input_embeddings=input_embeddings,
                                      key_padding_mask=encoder_key_padding_mask)


        decoder_outputs = self.decode(encoder_outputs=encoder_outputs,
                                      lebels_embeddings=lebels_embeddings,
                                      key_padding_mask=decoder_key_padding_mask,
                                      encoder_key_padding_mask=encoder_key_padding_mask)

        # TODO размерность labels_logits: [ ? ]
        labels_logits = self.decoder_labels_linear(decoder_outputs)

        # не забываем про смещение токенов для labels
        loss = self.criterion(labels_logits.reshape(-1, self.config.vocab_size),
                              labels[:, 1:].reshape(-1).long())

        return Seq2SeqLMOutput(
            loss=loss,
            decoder_hidden_states=decoder_outputs,
            encoder_hidden_states=encoder_outputs,
        )

    def generate(self, input_ids=None, attention_mask=None, token_type_ids=None, max_length=None, num_beams=None, **kwargs):
        """
        Метод используется в trainer.evaluate
        """

        batch_size = input_ids.shape[0]

        if attention_mask is not None:
            attention_mask = (attention_mask == 0)

        # надо
        # 1. Получить эмбэдддинги
        # 2. Получить Контекстные эмбэддинги (memory) через енкодер
        # 3. Запустить генерацию generate_encoded

        embeddings = self.embeddings(input_ids)
        rnn_outputs = self.encode(embeddings, attention_mask)
        predicted_tokens_sequences, _ = self.generate_encoded(batch_size=batch_size,
                                                              encoder_outputs=rnn_outputs,
                                                              encoder_key_padding_mask=attention_mask,
                                                              max_length=max_length,
                                                              num_beams=num_beams)

        return predicted_tokens_sequences

    def generate_encoded(self, batch_size=None, encoder_outputs=None, encoder_key_padding_mask=None, max_length=None, num_beams=None, **kwargs):

        generated_tokens = torch.tensor([self.config.bos_token_id] * batch_size, device=encoder_outputs.device).unsqueeze(1) # [ bs, 1 ]

        predicted_tokens_sequences = [ generated_tokens ] # [ bs, 1 ]


        # todo размерность decoder_outputs: [ ? ]
        decoder_outputs = []
        softmax_layer = nn.Softmax(dim=-1)

        for token_i in range(max_length):
            # генерим по одному токену
            # greedy decode

            # тут должны быть неконтекстные эмбэддинги для уже сгенерированных токенов
            generated_tokens_embeddings = self.embeddings(predicted_tokens_sequences[token_i])
            print('generated_tokens_embeddings', generated_tokens_embeddings.shape)
            print('encoder_outputs', encoder_outputs.shape)
            #print('key_padding_mask', key_padding_mask)
            #print('encoder_key_padding_mask', encoder_key_padding_mask.shape)
            print(self.subsequent_mask( generated_tokens_embeddings.shape[1] - 1,
                                       device=encoder_outputs.device))
            decoder_output = self.decode(encoder_outputs=encoder_outputs,
                                         lebels_embeddings=generated_tokens_embeddings,
                                          key_padding_mask=None,
                                          encoder_key_padding_mask=encoder_key_padding_mask)
            print('decoder_output', decoder_output.shape)
            last_token_outputs = decoder_output[:, -1:, :]
            decoder_outputs.append(last_token_outputs)

            next_token_logits = self.decoder_labels_linear(decoder_output)

            # из логитов надо получить вероятности токенов, как это делали в RNN
            next_generated_tokens = torch.max(next_token_logits, dim=2)[1]
            print('next_generated_tokens', next_generated_tokens)
            predicted_tokens_sequences.append(next_generated_tokens)
            # predicted_tokens_sequences расширили, поэтому надо обновить и generated_tokens
        generated_tokens = torch.cat(predicted_tokens_sequences, dim=-1)

        # это мы уже делали для RNN, делаем паддинг после EOS токена
        eos_token_indexes = torch.nonzero(generated_tokens == self.config.eos_token_id, as_tuple=False)
        for eos_idx in eos_token_indexes:
            generated_tokens[eos_idx[0], eos_idx[1]:] = self.config.pad_token_id

        return generated_tokens, decoder_outputs




In [59]:
transformer_attention_config = TransformerAttentionConfig(
    pad_token_id=fast_tokenizer.pad_token_id,
    bos_token_id=fast_tokenizer.bos_token_id,
    eos_token_id=fast_tokenizer.eos_token_id,
)
transformer_attention_model = Seq2SeqTransformerAttention(transformer_attention_config)

In [30]:
sum( p.numel() for p in transformer_attention_model.parameters() if p.requires_grad )

8897312

In [60]:
transformer_attention_model.generate( input_ids = torch.arange(15).reshape(3, 5), max_length=64 )

generated_tokens_embeddings torch.Size([3, 1, 128])
encoder_outputs torch.Size([3, 5, 128])
tensor([], size=(0, 0), dtype=torch.bool)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│ in generate:230                                                                                  │
│ in generate_encoded:261                                                                          │
│ in decode:164                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:369 in forward           │
│                                                                                                  │
│   366 │   │   output = tgt                                                                       │
│   367 │   │                                                                                      │
│   368 │   │   for mod in self.layers:                                                            │
│ ❱ 369 │   │   │   output = mod(output, memory, tgt_mask=tgt_mask,                                │
│   370 │   │   │   │   │   │    memory_mask=memory_mask,                                          │
│   371 │   │   │   │   │   │    tgt_key_padding_mask=tgt_key_padding_mask,                        │
│   372 │   │   │   │   │   │    memory_key_padding_mask=memory_key_padding_mask)                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:716 in forward           │
│                                                                                                  │
│   713 │   │   │   x = x + self._mha_block(self.norm2(x), memory, memory_mask, memory_key_paddi   │
│   714 │   │   │   x = x + self._ff_block(self.norm3(x))    

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = fast_tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, fast_tokenizer.pad_token_id)
    decoded_labels = fast_tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    print('decoded_preds[0]', fast_tokenizer.batch_decode(preds, skip_special_tokens=False)[0])
    print('decoded_labels[0]', decoded_labels[0])

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != fast_tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
transformer_attention_config = TransformerAttentionConfig(
    pad_token_id=fast_tokenizer.pad_token_id,
    bos_token_id=fast_tokenizer.bos_token_id,
    eos_token_id=fast_tokenizer.eos_token_id,
)
transformer_attention_model = Seq2SeqTransformerAttention(transformer_attention_config)


training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="steps",
    eval_steps=1000,
    learning_rate=1e-4,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1000,
    predict_with_generate=True,
    logging_steps=25,
)

trainer = Seq2SeqTrainer(
    model=transformer_attention_model,
    args=training_args,
    train_dataset=books_preprocessed["train"].select(range(256)),
    eval_dataset=books_preprocessed["train"].select(torch.tensor(range(256))),
    tokenizer=fast_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir my_awesome_opus_books_model/runs

## Протестируем модель

In [ ]:
# Сохраняем модель и токенайзер
fast_tokenizer.save_pretrained("./transformer_attention_tokenizer")
transformer_attention_model.save_pretrained("./transformer_attention_model")

In [ ]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq

# DataCollator отвечает за объединение данных в батчи -- добивает предложения до одной длинны (делает паддинг)
# преобразует numpy.array или питоновские списки в torch.Tensor

import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

loaded_tokenizer = AutoTokenizer.from_pretrained("./transformer_attention_tokenizer/")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = loaded_tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, loaded_tokenizer.pad_token_id)
    decoded_labels = loaded_tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != loaded_tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


rnn_config = TransformerAttentionConfig(
    pad_token_id=loaded_tokenizer.pad_token_id,
    bos_token_id=loaded_tokenizer.bos_token_id,
    eos_token_id=loaded_tokenizer.eos_token_id,
)

rnn_attention_model_loaded = Seq2SeqTransformerAttention(rnn_config).from_pretrained("./transformer_attention_model/", local_files_only=True)

training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="steps",
    eval_steps=1000,
    learning_rate=1e-4,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1000,
    predict_with_generate=True,
    logging_steps=25,
)

data_collator = DataCollatorForSeq2Seq(tokenizer=loaded_tokenizer, return_tensors="pt")


trainer = Seq2SeqTrainer(
    model=transformer_attention_model,
    args=training_args,
    eval_dataset=books_preprocessed["train"].select(range(256)),  # валидировать будем тоже на обучающих данных (дисклаймер: это можно делать только для тестирования)
    tokenizer=fast_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


evaluate_result = trainer.evaluate(test_dataset=books_preprocessed["train"].select(range(256)))

print("evaluate_result", evaluate_result)

assert evaluate_result['eval_bleu'] >= 50

Если ассерт выше прошел, можете загрузить ноутбук и обученные модельки+конфиги на гитхаб. На гитхабе из ноутбука будет выгружена ячейка с описанием модели и будет запускаться тест аналогичный ячейке выше.

Веса и конфиги надо заархивировать и на гитхаб загрузить в виде архива с сохранением названий: `rnn_tokenizer.zip`, `rnn_attention_model.zip`

Ноутбук надо загрузить с таким же названием, какое было в репозитории.


Важно! Архив с весами и конфигом модели не должен весить больше 100МБ иначе этот файлик не будет скачан в пайплайнах даже если вы его загрузите на гитхаб через git-lfs

In [ ]:
!zip -r transformer_attention_model.zip transformer_attention_model
!zip -r transformer_attention_tokenizer.zip transformer_attention_tokenizer
!ls -ltrh | tail -n2

Мы зааихивировали конфиги и веса, скачайте архивы из колаба и загрузите в github репозиторий. **Не забудьте обновить ноутбук в github репозитории тоже!**


Архив с моделькой может занимтаь больше 25 мегабайт. Гитхаб не разрешает грузить большие файлы через веб-интерфейс. Можете закоммитить эти архивы через консоль или погуглить, как это сделать по-другому
https://bytesbin.com/upload-files-larger-than-25mb-to-github/
https://www.google.com/search?q=Fix+GitHub+%E2%80%98Yowza+That%E2%80%99s+a+Big+File%E2%80%99

## Протестируйте модель в pipeline, будет ли она давать предсказания для предложений, которых не было в обучающем датасете?

In [ ]:
from transformers import pipeline

# todo

# Вопросы!

## Почему статья называется "Attention Is All You Need"?

## Почему обучение декодера RNN нельзя распараллелить, как это делается Transformer Decoder с помощью subsequent mask?